In [1]:
%pip install python-dotenv psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import psycopg2
from dotenv import load_dotenv
# ... (other imports)

# load_dotenv()

# USER=os.getenv("USER")
# PASSWORD=os.getenv("PASSWORD")
# HOST=os.getenv("HOST")
# PORT=5432
# DBNAME=os.getenv("DBNAME")

USER="postgres" 
PASSWORD = "itspgpassword@2025"
HOST = "db.kcsdqmtsdasqlancgygk.supabase.co" # Use the correct format here
PORT="5432"
DBNAME = "postgres"



# Connect to the database
try:
    connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME,
        sslmode='require' 
    )
    print("Connection successful! Supabase credentials validated.")

except Exception as e:
    # If this fails, the issue is still a typo in HOST or a network firewall blocking port 5432.
    print(f"Failed to connect: {e}")

Connection successful! Supabase credentials validated.


In [3]:
cur = connection.cursor()

In [4]:
QUERY = "SELECT product_name, price, category FROM product_details LIMIT 3;" 
    
# 3. Execute the query
cur.execute(QUERY)
# 4. Fetch all results
results = cur.fetchall()

In [5]:
results

[("Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2",
  '257',
  'Personal Care'),
 ('Care Cosmetics Apple Day Cream with Apple juice to improve skin complexion - 50gm - Pack of 2',
  '386',
  'Personal Care'),
 ("Care Cosmetics Care's Chicknut Face Pack with Chikoo for sensitive skin - 60gm - Pack of 2",
  '460',
  'Personal Care')]

In [6]:
connection.commit()

In [7]:
connection.rollback()

In [8]:
cur.execute("select id,product_name,category,price,rating,available from product_details LIMIT 3;")
results = cur.fetchall()
connection.commit()
results

[(1,
  "Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2",
  'Personal Care',
  '257',
  'N/A',
  'yes'),
 (2,
  'Care Cosmetics Apple Day Cream with Apple juice to improve skin complexion - 50gm - Pack of 2',
  'Personal Care',
  '386',
  'N/A',
  'no'),
 (3,
  "Care Cosmetics Care's Chicknut Face Pack with Chikoo for sensitive skin - 60gm - Pack of 2",
  'Personal Care',
  '460',
  'N/A',
  'yes')]

In [9]:
type(results[0])

tuple

In [10]:
%pip install langchain langchain-google-genai langchain-community sqlalchemy python-dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Install necessary packages if not already installed
# Uncomment if needed:
# !pip install langchain langchain-google-genai langchain-community sqlalchemy python-dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

c:\Users\kumar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- Load environment variables (optional if using .env) ---
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_DB_URL = os.getenv("SUPABASE_DB_URL")

In [13]:
# --- Initialize the Gemini model ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

In [14]:
llm.invoke("Hello").content

'Hello! How can I help you today?'

In [15]:
try:
    engine = create_engine(SUPABASE_DB_URL)
    db = SQLDatabase(
        engine=engine,
        include_tables=['product_details']  # only this table for now
    )
    print("✅ Database connection successful.")
except Exception as e:
    print(f"❌ Error connecting to DB: {e}")


✅ Database connection successful.


In [16]:
CUSTOM_SYSTEM_PREFIX = """
You are a highly skilled Text-to-SQL agent for a personal care products company.
You have access to a PostgreSQL database containing product information.
Your goal is to accurately convert user questions into executable SQL queries, execute them, and then provide a friendly, conversational answer based on the results.

Rules:
1. **Always** use the provided database schema (tables: product_details, chat_history).
2. For keyword searches, use efficient PostgreSQL filtering methods.
3. **NEVER** use INSERT, UPDATE, DELETE, or DROP commands. Only SELECT queries are allowed.
4. Limit results to 10 rows maximum.
5. If the user asks a non-product question, respond politely without SQL execution.

Begin!
"""


In [17]:
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=CUSTOM_SYSTEM_PREFIX,
    verbose=True,
    handle_parsing_errors=True
)
print("✅ Agent successfully created.")

✅ Agent successfully created.


In [18]:
def get_product_info(user_query: str):
    """Executes Text-to-SQL flow for user queries."""
    print(f"\n--- Processing User Query: '{user_query}' ---")
    try:
        response = agent_executor.invoke({"input": user_query})
        return response['output']
    except Exception as e:
        return f"An error occurred during query execution: {e}"


In [19]:
user_question_1 = "whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant."
answer_1 = get_product_info(user_question_1)
print(f"\n💬 Chatbot Response:\n{answer_1}")



--- Processing User Query: 'whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant.' ---


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:product_detailsI need to find the benefits of "Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant". I will query the `product_details` table for this information.
Action: sql_db_schema
Action Input: product_details
CREATE TABLE product_details (
	product_name TEXT, 
	category TEXT, 
	price TEXT, 
	rating TEXT, 
	image_url TEXT, 
	product_url TEXT, 
	available TEXT, 
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	CONSTRAINT product_details_pkey PRIMARY KEY (id), 
	CONSTRAINT product_details_id_key UNIQUE NULLS DISTINCT (id)
)

/*
3 rows from product_details table:
product_name	category	price	rating	image_url	product_url	available	id
Care Cosmetics Care's Ant

In [21]:
print(f"\n💬 Chatbot Response:\n{answer_1}")


💬 Chatbot Response:
The Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant is priced at 221 and falls under the 'Personal Care' category.


In [23]:
llm.invoke("whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant").content

'Medibar Personal Lubricant, especially with its "Long Lasting Pleasure" claim, offers several key benefits for both men and women:\n\n1.  **Enhanced Comfort and Reduced Friction:**\n    *   **Prevents Discomfort:** The primary benefit of any lubricant is to reduce friction during sexual activity. This prevents chafing, irritation, and soreness that can occur due to insufficient natural lubrication.\n    *   **Smoother Movement:** It allows for more fluid and comfortable movement, making penetration easier and more enjoyable for both partners.\n\n2.  **Combats Natural Dryness:**\n    *   **Addresses Vaginal Dryness:** Many women experience vaginal dryness due to various factors like hormonal changes (menopause, postpartum), certain medications, stress, or even just insufficient arousal. Medibar helps to supplement natural lubrication, making intercourse comfortable and pleasurable again.\n    *   **For Men:** While less common, some men can also benefit from lubrication, especially dur